In [5]:
!pip install emoji
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 9.6 MB/s eta 0:00:00


In [14]:
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import emoji
import contractions
import re
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [15]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [39]:
model = load_model('/content/drive/MyDrive/Colab Notebooks/Sentiment Analysis/final_model.keras')
with open('/content/drive/MyDrive/Colab Notebooks/Sentiment Analysis/tokenizer.pkl','rb') as handle:
  tokenizer = pickle.load(handle)

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 13 variables whereas the saved optimizer has 24 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [9]:
def convert_emoji_to_text(text):
  return emoji.demojize(str(text), delimiters=(' ',' '))

In [10]:
def expand_contractions(text):
    return contractions.fix(text)

In [11]:
slang_dict = {
    "lol": "laughing out loud",
    "omg": "oh my god",
    "idk": "I do not know",
    "tbh": "to be honest",
    'rofl': 'rolling on floor laughing',
    'xo':'hugs and kisses',
    'XD':'Laugh',
    'hehe':'laughter',
    'aww':'cute',
    'nah':'no',
    'rip':'rest in peace',
    'wtf':'what the fuck',
    'oml':'oh my laughing'
}

def expand_slang(text):
    for slang, full in slang_dict.items():
        text = text.replace(slang, full)
    return text

In [13]:
def cleaner(text):
  text = re.sub(r'http\S+|www.\S+|https\S+', '', text) #URLs
  text = re.sub(r'@\w+','',text) #Mentions
  text = re.sub(r'#','',text) #Hashtags
  text = re.sub(r'[^a-zA-z\s]','',text) #Special characters/numbers
  text = re.sub(r'\s+', ' ', text).strip()
  text = text.lower()
  return text

In [17]:
custom_stopwords = ['rt', 'via', 'amp', 'u', 'im', 'ur']
stop_words = set(stopwords.words('english'))
stop_words.update(custom_stopwords)
stop_words.discard('not')
stop_words.discard('no')
stop_words.discard('yes')
lemmatizer = WordNetLemmatizer()

In [32]:
def prepare(data):
  data = convert_emoji_to_text(data)
  data = expand_contractions(data)
  data = expand_slang(data)
  data = cleaner(data)
  token = word_tokenize(data)
  filtered = [word for word in token if word not in stop_words]
  lemmatized = [lemmatizer.lemmatize(word) for word in filtered]
  final = ' '.join(lemmatized)
  return final

In [33]:
texts  = 'Hello I am running a marathon finally yes'
final = prepare(texts)
final

'hello running marathon finally yes'

In [41]:
def pred_sentiment(sentence, max_length=50):
  cleaned = prepare(sentence)
  sequence = tokenizer.texts_to_sequences([cleaned])
  padded = pad_sequences(sequence,maxlen=max_length, padding='post', truncating='post')
  prob = model.predict(padded)[0][0]
  sentiment = "Positive" if prob >= 0.5 else "Negative"
  print(f"Sentiment: {sentiment} (probability={prob:.2f})")
  return sentiment, prob

In [47]:
while True:
    user_input = input("Type a sentence (or 'quit' to exit): ")
    if user_input.lower() == 'quit':
        break
    pred_sentiment(user_input)

Type a sentence (or 'quit' to exit): This isn't just a beautifully crafted gangster film. Or an outstanding family portrait, for that matter. An amazing period piece. A character study. A lesson in filmmaking and an inspiration to generations of actors, directors, screenwriters and producers. For me, this is more: this is the definitive film. 10 stars out of 10.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Sentiment: Positive (probability=0.66)
Type a sentence (or 'quit' to exit): 'm sure it was good for its time, but by todays standard this is truly mediocre. It's 3 hours of boredom, cheesy stereotypical dialog, and unrealistic mafia. Some of the sub plots, like the 30 minute Sicilia sequence, with Michael marrying this girl we have never seen before, only for the movie to kill her off 20 minutes later, before we've even heard her talk? One of the most useless sub plots I have ever seen in any movie. Did they expect us to care for her after introducing her 20 minutes ago, and not even giving